In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import time
from pprint import pprint
import requests
from datetime import date, timedelta, datetime
import json
# Import API key
from config import weather_api_key
from pprint import pprint

### Get Long Lat list of tuples to pass into Weather API

In [ ]:
geojson_data = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/4.5_month.geojson'

In [ ]:
response = requests.get(geojson_data).json()
features = response["features"]
test = features[0]['properties']
coords_list = []
for data in features:
    time = data['properties']['time']
    magnitude = data['properties']['mag']
    place = data['properties']['place']
    
    long_coords = data["geometry"]["coordinates"][0]
    lat_coords = data["geometry"]["coordinates"][1]
    
    coords = {
            'latlng': f"{lat_coords}, {long_coords}",
            'lat': lat_coords,
            'long': long_coords,
            'time': time,
            'magnitude': magnitude,
            'place': place}
    
    coords_list.append(coords)
# pprint(coords_list)
# coords_list

In [ ]:
len(coords_list)

-----

### Use Coordinates to get historical weather data.
Coordinates represent earthquake lat long location

This section creates the table WeatherSeries in the database. WeatherSeries has 4 day weather forecast and matching earthquake data.

In [ ]:
from tqdm import tqdm
from tqdm import tqdm_notebook

In [ ]:
pbar = tqdm(total=len(coords_list))

tuple_of_data = []
add_tuple = {}
counter = 0

def checkForTwo(number):
#     if (int(number)) <= 1:
#         number = '1'
    if (int(number)) < 10:
        number = "0" + str(number)
    else:
#         print(number)
        number = str(number)
        pass
    return number

for coordinates in coords_list:
    # Lat / long
    coords = coordinates['latlng']
    
    ######################
    ## Earthquake time conversion
    ######################
    earthquake_time = coordinates['time']
    dt3 = datetime.fromtimestamp(earthquake_time / 1000)
    # Time range end is the day of the earthquake
    time_range_end = str(dt3.year) + "-" + checkForTwo(dt3.month) + "-" + checkForTwo(dt3.day)
    
    # Time range begin is 3 days before the earthquake
    four_day_series = dt3 - timedelta(3)
    four_day_day = four_day_series
    time_range_begin = str(dt3.year) + "-" + checkForTwo(dt3.month) + "-" + checkForTwo(four_day_day.day)
    ######################
    
    base_url = 'http://api.weatherstack.com/historical'
    params_weather = {'access_key': weather_api_key, 
                    'query': coords,
                    'historical_date_start': time_range_begin,
                    'historical_date_end': time_range_end,
                    'units': 'f'
                 }
    
    response = requests.get(base_url, params=params_weather).json()
    # url_link = requests.get(base_url, params=params_weather).url
    
    try: 
        # extract results
        location = response.get('location')
        # City, Country, Region
        city = location["name"]
        country = location["country"]
        region = location["region"]

        # Historical Only
        historical = response['historical']
#         print(region)
        for hist in historical:
            add_tuple = {
                'city': city,
                'country': country,
                'region': region,
                'avgtemp': historical[hist]['avgtemp'],
                'date': historical[hist]['date'],
                'date_epoch': historical[hist]['date_epoch'],
                'maxtemp': historical[hist]['maxtemp'],
                'mintemp': historical[hist]['mintemp'],
                'sunhour': historical[hist]['sunhour'],
                'totalsnow': historical[hist]['totalsnow'],
                'uv_index': historical[hist]['uv_index'],
                'long': coordinates['long'],
                'lat': coordinates['lat'],
                'time': coordinates['time'],
                'magnitude': coordinates['magnitude'],
                'place': coordinates['place']
            }
            tuple_of_data.append(add_tuple)
            pbar.update(1)
    except TypeError as e:

    # print(e)
        continue
    pbar.update(1)
    counter += 1

pbar.close() 

# tuple_of_data

In [ ]:
len(tuple_of_data)

## Saving to JSON

In [ ]:
from datetime import datetime
now = datetime.now()

In [ ]:
# Get time
timedate = now.strftime("%Y-%m-%d_%H_%M_%S")

# SAVE: Player_position
with open(f'../{timedate}_4_day_weather_for_earthquakes.json', 'w') as fp:
    json.dump(tuple_of_data, fp)
    

In [ ]:
# del tuple_of_data

## Opening JSON .. Do not have to run the api again.

In [ ]:
# Open raw json to parse
with open('../2020-08-17_15_15_40_4_day_weather_for_earthquakes.json', 'r') as f:
    weather_earthquake_data = json.load(f)

In [ ]:
# Dataframe to see raw(cleaned) data
all_logs = pd.DataFrame(weather_earthquake_data)
# all_logs.head()

## Upload to PostgreSQL

In [ ]:
# Use this to clear out the db
# ----------------------------------
# # Session.rollback(self)
# Base.metadata.drop_all(engine)
# session.commit()

In [ ]:
# Delete previous entries
# del WeatherSeries
# del LatLong

In [2]:
# Dependencies and Setup
import pandas as pd
# import numpy as np
# import time

# import requests
from datetime import date, timedelta, datetime
import json
# Import API key
# from config import weather_api_key
# from pprint import pprint

In [3]:
# Dependencies
# ----------------------------------
# Imports the method used for connecting to DBs
from sqlalchemy import create_engine
# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Text, DateTime, Float, Boolean, ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session

In [4]:
# Create Weather and Earthquake Classes
# ----------------------------------
# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()

In [5]:
## Class base template to upload to sqlite
class WeatherSeries(Base):
    __tablename__ = 'weatherSeries'

    id = Column(Integer, primary_key=True)
    city = Column(String(50))
    country = Column(String(200))
    region = Column(String(80))
    avgtemp = Column(Float)
    date = Column(String(12))
    date_epoch = Column(Float)
    maxtemp = Column(Float)
    mintemp = Column(Float)
    sunhour = Column(Float)
    totalsnow = Column(Float)
    uv_index = Column(Float)
    magnitude = Column(Float)
    place = Column(String(80))
    lat = Column(String(12))
    long = Column(String(12))
    
# class LatLong(Base):
#     __tablename__ = 'latlong'
#     id = Column(Integer, primary_key=True)
#     lat = Column(String(12))
#     long = Column(String(12))
#     lat_rel = Column(Integer, ForeignKey('weatherSeries.id'))
#     weatherSer = relationship(WeatherSeries, primaryjoin=lat_rel == WeatherSeries.id)
#     latlong_id = Column(String,ForeignKey('latlong.id'))
#     latlong_rel = relationship(LatLong)
#     lat = Column(String(12))
#     long = Column(String(12))

In [6]:
# Create Database Connection
# ----------------------------------
# Creates a connection to our DB
# Engine opens the door. Conn is the walk through sign
engine = create_engine("sqlite:///earthquake_weather.sqlite")
conn = engine.connect()
# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine)
# Create a Session Object to Connect to DB
# ----------------------------------
session = Session(bind=engine)

In [ ]:
# ## Add weather series to sql
# x=0
# while x <= (len(all_logs) - 1):    
#     lat_long_data = LatLong(
#         lat = all_logs["lat"][x],
#         long = all_logs["long"][x]
#         )
#     x+=1
#     # Add Records to the Appropriate DB
#     # ----------------------------------
#     # Use the SQL ALchemy methods to run simple "INSERT" statements using the classes and objects  
#     session.add(lat_long_data)
#     # session.add(earthquake_data)
#     session.commit()
# print('Complete: Uploaded to SQLite DB')

In [ ]:
## Function: Add Weather Series to sql
def addToSQL(all_logs):
    x=0
    while x <= (len(all_logs) - 1):
        try: 
            weather_data = WeatherSeries(
                city = all_logs["city"][x],
                country = all_logs["country"][x],
                region = all_logs["region"][x],
                avgtemp = all_logs["avgtemp"][x],
                date = all_logs["date"][x],
                date_epoch = all_logs["date_epoch"][x],
                maxtemp = all_logs["maxtemp"][x],
                mintemp = all_logs["mintemp"][x],
                sunhour = all_logs["sunhour"][x],
                totalsnow = all_logs["totalsnow"][x],
                uv_index = all_logs["uv_index"][x],
                magnitude = all_logs["magnitude"][x],
                place = all_logs["place"][x],
                lat = all_logs["lat"][x],
                long = all_logs["long"][x],
                )
            x+=1
        except OperationalError:
            pass
            
        # Add Records to the Appropriate DB
        # ----------------------------------
        # Use the SQL ALchemy methods to run simple "INSERT" statements using the classes and objects  
        session.add(weather_data)
        # session.add(earthquake_data)
        session.commit()
    return 'Complete: Uploaded to SQLite DB'

In [ ]:
addToSQL(all_logs)

----

## Query from DB

DataFrame 

In [7]:
# Verify Data 
# Create DataFrame from sql table Weather
weather_data_df = pd.read_sql("SELECT * FROM WeatherSeries", conn)
weather_data_df.head()

,id,city,country,region,avgtemp,date,date_epoch,maxtemp,mintemp,sunhour,totalsnow,uv_index,magnitude,place,lat,long
0,1,El Azufre,Mexico,Oaxaca,84.0,2020-08-14,1.597363e+09,88.0,77.0,10.2,0.0,7.0,4.8,"127 km SSW of Santa María Chico Ometepec, Mexico",15.0758,-98.3289
1,2,El Azufre,Mexico,Oaxaca,82.0,2020-08-15,1.597450e+09,86.0,77.0,10.2,0.0,6.0,4.8,"127 km SSW of Santa María Chico Ometepec, Mexico",15.0758,-98.3289
2,3,El Azufre,Mexico,Oaxaca,82.0,2020-08-16,1.597536e+09,86.0,77.0,6.2,0.0,6.0,4.8,"127 km SSW of Santa María Chico Ometepec, Mexico",15.0758,-98.3289
3,4,El Azufre,Mexico,Oaxaca,82.0,2020-08-17,1.597622e+09,86.0,75.0,6.2,0.0,6.0,4.8,"127 km SSW of Santa María Chico Ometepec, Mexico",15.0758,-98.3289
4,5,Las Enramadas,Nicaragua,Chinandega,84.0,2020-08-14,1.597363e+09,90.0,77.0,10.6,0.0,7.0,4.8,"59 km S of Intipucá, El Salvador",12.6645,-88.1243


In [9]:
# avg = weather_data_df[weather_data_df['magnitude'] > 6.0]
# avg

#### Python call from db. For flask to web

In [48]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
# Go to existing database with automap_base
from sqlalchemy.ext.automap import automap_base
# Work through mapper to use python code
from sqlalchemy.orm import Session
# Inspect with python
from sqlalchemy import create_engine, inspect
from sqlalchemy import desc


In [49]:
Base = automap_base()
Base.prepare(engine, reflect=True)
# Check db table names
# Base.classes.keys()
weather_table = Base.classes.weatherSeries

## analysisChartCall

In [56]:
def analysisChart():
    Base = automap_base()
    Base.prepare(engine, reflect=True)
    weather_table = Base.classes.weatherSeries
    analysis_container = session.query(weather_table).order_by(desc(weather_table.date)).all()        
    analysis_list_temp = []
    x=1

    for data in analysis_container:
        # get specific data from db
        container = {
            "date": data.date,
            "magnitude": data.magnitude,
            "maxtemp": data.maxtemp,
            "mintemp": data.mintemp,
    #         "avgtemp": data.avgtemp,
            "lat": data.lat,
            }
        analysis_list_temp.append(container)

    # Create df for parsing    
    temp_df = pd.DataFrame(analysis_list_temp)
    # Sort by lat and date, reset index
    temp_df = temp_df.sort_values(by=['lat', 'date'], ascending=False).reset_index(drop=True)

    # Make copy of df, remove 2nd and 3rd log keeping 1st and 4th log of one eq entry.
    run_df = temp_df.copy()
    while x < len(temp_df.index):
        run_df=run_df.drop(x)
        x+=1
        run_df=run_df.drop(x)
        x+=3

    # Reset index 
    run_df = run_df.reset_index(drop=True) 

    # get difference of weather change from day of eq and few days before
    i = 0
    new_col = []
    # Icon list will tell style which icon to display
    icon_list = []
    while i < len(run_df.index):
    #     for data in run_df.index:
        first = run_df.iloc[i,2]
        second = run_df.iloc[i+1, 2]
        difference = first - second
        new_col.append(difference)
        new_col.append(difference)
        i+=2    
    # Add new list to df as a new column  
    run_df['difference'] = new_col     
    # Count up, nochange, down
    up_count = 0
    nochange_count = 0
    down_count = 0
    for x in run_df['difference']:
        if x > 0:
            icon = "up"
            up_count+=1
            icon_list.append(icon)
        elif x == 0:
            icon = "nochange"
            nochange_count+=1
            icon_list.append(icon)
        else:
            icon = "down"
            down_count+=1
            icon_list.append(icon)

    # Add new list to df as a new column
    run_df['icon'] = icon_list    
    # select only the columns we need
    run_df = run_df[['date','magnitude','lat','difference','icon']]

    # # Turn df into list of tuples
    records = run_df.to_records(index=False)
    analysis_chart = list(records)

    # Create list of tuple
    analysis_list = []
    for data in analysis_chart:
        container2 = {
            "date": data.date, 
            "magnitude": data.magnitude, 
            "lat": data.lat, 
            "difference": data.difference, 
            "icon": data.icon,
            }
        analysis_list.append(container2)

    diff_count = len(run_df['difference'])
    above_percentage = "{:.0%}".format(up_count / diff_count)
    atzero_percentage = "{:.0%}".format(nochange_count / diff_count)
    belowzero_percentage = "{:.0%}".format(down_count / diff_count)
    container3 = {
                "abovezero": up_count,
                "abovezeropercent": above_percentage,
                "atzero": nochange_count, 
                "atzeropercent": atzero_percentage, 
                "belowzero": down_count, 
                "belowzeropercent": belowzero_percentage,
                }

    analysis_list.append(container3)     
    return analysis_list
analysisChart()

[{'date': '2020-07-18',
  'magnitude': 4.8,
  'lat': '9.5869',
  'difference': -2.0,
  'icon': 'down'},
 {'date': '2020-07-15',
  'magnitude': 4.8,
  'lat': '9.5869',
  'difference': -2.0,
  'icon': 'down'},
 {'date': '2020-07-29',
  'magnitude': 4.5,
  'lat': '9.0332',
  'difference': 4.0,
  'icon': 'up'},
 {'date': '2020-07-26',
  'magnitude': 4.5,
  'lat': '9.0332',
  'difference': 4.0,
  'icon': 'up'},
 {'date': '2020-07-27',
  'magnitude': 5.8,
  'lat': '9.0127',
  'difference': 0.0,
  'icon': 'nochange'},
 {'date': '2020-07-24',
  'magnitude': 5.8,
  'lat': '9.0127',
  'difference': 0.0,
  'icon': 'nochange'},
 {'date': '2020-07-30',
  'magnitude': 5.7,
  'lat': '8.5416',
  'difference': 6.0,
  'icon': 'up'},
 {'date': '2020-07-27',
  'magnitude': 5.7,
  'lat': '8.5416',
  'difference': 6.0,
  'icon': 'up'},
 {'date': '2020-07-29',
  'magnitude': 4.5,
  'lat': '8.4405',
  'difference': -2.0,
  'icon': 'down'},
 {'date': '2020-07-26',
  'magnitude': 4.5,
  'lat': '8.4405',
  'diff

NameError: name 'analysis_chart' is not defined

In [46]:
diff_count = len(run_df['difference'])/2
diff_count

373.0

In [40]:
run_df['icon'].value_counts()


TypeError: 'numpy.int64' object is not callable

In [11]:
run_df.sort_values(by='difference')

,date,magnitude,lat,difference,icon
255,2020-07-17,4.9,30.4051,-14.0,down
254,2020-07-20,4.9,30.4051,-14.0,down
243,2020-07-17,4.9,30.4402,-14.0,down
242,2020-07-20,4.9,30.4402,-14.0,down
180,2020-08-06,4.9,36.5497,-12.0,down
...,...,...,...,...,...
263,2020-07-21,4.9,30.3884,12.0,up
297,2020-08-07,4.5,28.9031,13.0,up
296,2020-08-10,4.5,28.9031,13.0,up
609,2020-08-07,4.5,-23.9032,15.0,up


In [66]:
def analysisChartCall():
    # Sets an object to utilize the default declarative base in SQL Alchemy
    Base = declarative_base()
    ## Class base template to upload to sqlite
    class WeatherSeries(Base):
        __tablename__ = 'weatherSeries'

        id = Column(Integer, primary_key=True)
        city = Column(String(50))
        country = Column(String(200))
        region = Column(String(80))
        avgtemp = Column(Float)
        date = Column(String(12))
        date_epoch = Column(Float)
        maxtemp = Column(Float)
        mintemp = Column(Float)
        sunhour = Column(Float)
        totalsnow = Column(Float)
        uv_index = Column(Float)
        magnitude = Column(Float)
        place = Column(String(80))
        lat = Column(String(12))
        long = Column(String(12))

    # Create Database Connection
    # ----------------------------------
    # Creates a connection to our DB
    # Engine opens the door. Conn is the walk through sign
    engine = create_engine("sqlite:///earthquake_weather.sqlite")
    conn = engine.connect()
    # Create a "Metadata" Layer That Abstracts our SQL Database
    # ----------------------------------
    # Create (if not already in existence) the tables associated with our classes.
    Base.metadata.create_all(engine)
    # Create a Session Object to Connect to DB
    # ----------------------------------
    session = Session(bind=engine)

    def analysisChart():
        Base = automap_base()
        Base.prepare(engine, reflect=True)
        weather_table = Base.classes.weatherSeries
        analysis_container = session.query(weather_table).order_by(desc(weather_table.date)).all()        
        analysis_list_temp = []
        x=1

        for data in analysis_container:
            # get specific data from db
            container = {
                "date": data.date,
                "magnitude": data.magnitude,
                "maxtemp": data.maxtemp,
                "mintemp": data.mintemp,
        #         "avgtemp": data.avgtemp,
                "lat": data.lat,
                }
            analysis_list_temp.append(container)

        # Create df for parsing    
        temp_df = pd.DataFrame(analysis_list_temp)
        # Sort by lat and date, reset index
        temp_df = temp_df.sort_values(by=['lat', 'date'], ascending=False).reset_index(drop=True)

        # Make copy of df, remove 2nd and 3rd log keeping 1st and 4th log of one eq entry.
        run_df = temp_df.copy()
        while x < len(temp_df.index):
            run_df=run_df.drop(x)
            x+=1
            run_df=run_df.drop(x)
            x+=3

        # Reset index 
        run_df = run_df.reset_index(drop=True) 

        # get difference of weather change from day of eq and few days before
        i = 0
        new_col = []
        # Icon list will tell style which icon to display
        icon_list = []
        while i < len(run_df.index):
        #     for data in run_df.index:
            first = run_df.iloc[i,2]
            second = run_df.iloc[i+1, 2]
            difference = first - second
            new_col.append(difference)
            new_col.append(difference)
            i+=2    
        # Add new list to df as a new column  
        run_df['difference'] = new_col     
        # Count up, nochange, down
        up_count = 0
        nochange_count = 0
        down_count = 0
        for x in run_df['difference']:
            if x > 0:
                icon = "up"
                up_count+=1
                icon_list.append(icon)
            elif x == 0:
                icon = "nochange"
                nochange_count+=1
                icon_list.append(icon)
            else:
                icon = "down"
                down_count+=1
                icon_list.append(icon)

        # Add new list to df as a new column
        run_df['icon'] = icon_list    
        # select only the columns we need
        run_df = run_df[['date','magnitude','lat','difference','icon']]

        # # Turn df into list of tuples
        records = run_df.to_records(index=False)
        analysis_chart = list(records)

        # Create list of tuple
        analysis_list = []
        for data in analysis_chart:
            container2 = {
                "date": data.date, 
                "magnitude": data.magnitude, 
                "lat": data.lat, 
                "difference": data.difference, 
                "icon": data.icon,
                }
            analysis_list.append(container2)

        diff_count = len(run_df['difference'])
        above_percentage = "{:.0%}".format(up_count / diff_count)
        atzero_percentage = "{:.0%}".format(nochange_count / diff_count)
        belowzero_percentage = "{:.0%}".format(down_count / diff_count)
        container3 = {
                    "abovezero": up_count,
                    "abovezeropercent": above_percentage,
                    "atzero": nochange_count, 
                    "atzeropercent": atzero_percentage, 
                    "belowzero": down_count, 
                    "belowzeropercent": belowzero_percentage,
                    }

        analysis_list.append(container3)
        return analysis_list
    analysis_list = analysisChart()
    return analysis_list


In [68]:
test = analysisChartCall()[-1]
test

{'abovezero': 266,
 'abovezeropercent': '36%',
 'atzero': 200,
 'atzeropercent': '27%',
 'belowzero': 280,
 'belowzeropercent': '38%'}

In [ ]:
#################################################################
## Facts 
##################################################################

def aboveSixQuakeCall():
    # Sets an object to utilize the default declarative base in SQL Alchemy
    Base = declarative_base()
    ## Class base template to upload to sqlite
    class WeatherSeries(Base):
        __tablename__ = 'weatherSeries'

        id = Column(Integer, primary_key=True)
        city = Column(String(50))
        country = Column(String(200))
        region = Column(String(80))
        avgtemp = Column(Float)
        date = Column(String(12))
        date_epoch = Column(Float)
        maxtemp = Column(Float)
        mintemp = Column(Float)
        sunhour = Column(Float)
        totalsnow = Column(Float)
        uv_index = Column(Float)
        magnitude = Column(Float)
        place = Column(String(80))
        lat = Column(String(12))
        long = Column(String(12))

    # Create Database Connection
    # ----------------------------------
    # Creates a connection to our DB
    # Engine opens the door. Conn is the walk through sign
    engine = create_engine("sqlite:///earthquake_weather.sqlite")
    conn = engine.connect()
    # Create a "Metadata" Layer That Abstracts our SQL Database
    # ----------------------------------
    # Create (if not already in existence) the tables associated with our classes.
    Base.metadata.create_all(engine)
    # Create a Session Object to Connect to DB
    # ----------------------------------
    session = Session(bind=engine)

    def aboveSixQuake():
        Base = automap_base()
        Base.prepare(engine, reflect=True)
        # Check db table names
        # Base.classes.keys()
        weather_table = Base.classes.weatherSeries
        
#         weather_container = session.query(weather_table).order_by(desc(weather_table.magnitude)).limit(1).all()
        
        weather_container = session.query(weather_table).filter(weather_table.magnitude > 6).all()
        weather_facts = []
        magnitude_list = []
        count = 0
        for data in weather_container:
            count += 1
            
            # make a list of magnitudes recorded greater than 6 and get avg temp
            magnitude_list.append(data.avgtemp)
            magnitude = data.magnitude
            magnitude_keep = 6
            # Get highest recorded earthquake
            if data.magnitude > magnitude_keep:
                magnitude_keep = data.magnitude
                location = data.country
                city = data.city
                temp_low = data.mintemp
                temp_high = data.maxtemp
                avg_temp_at_time = data.avgtemp
                date = data.date
            else:
                continue



        def spellDate(datestring):
            date_time_obj = datetime.datetime.strptime(datestring, '%Y-%m-%d')
            month_name = date_time_obj.strftime("%B")
            day = date_time_obj.strftime("%d")
            year = date_time_obj.strftime("%Y")

            month_day = month_name + " " + day
            month_day_year = month_name + " " + day + ", " + year

            date = {
                "month_day": month_day,
                "month_day_year": month_day_year,
            }
            return date  

  
        # Get avgtemp from list        
        def Average(lst): 
            return sum(lst) / len(lst) 
        quake_avg = Average(magnitude_list)


        spell_dates = spellDate(date)
        
        container = {
            "count": count, 
            "avgtemp": quake_avg,
            "highest_magnitude": magnitude_keep, 
            "highest_city": city,
            "highest_location": location,
            "temp_low": temp_low,
            "temp_high": temp_high,
            "avg_temp_at_time": avg_temp_at_time,
            "date": spell_dates,
            
        }
        weather_facts.append(container)
        return weather_facts

    weather_facts = aboveSixQuake()

    # Return results
    return weather_facts



In [ ]:
def latestQuakes():
    Base = automap_base()
    Base.prepare(engine, reflect=True)

    weather_table = Base.classes.weatherSeries
    weather_container = session.query(weather_table).order_by(desc(weather_table.date)).limit(5).all()

    weather_latest5 = []
    for data in weather_container:
        container = {
            "date": data.date, 
            "city": data.city, 
            "country": data.country, 
            "region": data.region, 
            "place": data.place,
            "mintemp": data.mintemp, 
            "maxtemp": data.maxtemp, 
            "avgtemp": data.avgtemp,
            "magnitude": data.magnitude, 
            }
        weather_latest5.append(container)
    return weather_latest5
    #     print(container)

In [ ]:
last_five_quakes_df = pd.DataFrame(last_five_quakes)
last_five_quakes_df

In [ ]:
# def weatherTimeSeries(query_call):
#     Base = automap_base()
#     Base.prepare(engine, reflect=True)
#     # Check db table names
#     # Base.classes.keys()
#     weather_table = Base.classes.weatherSeries
#     weather_container = session.query(weather_table).filter(weather_table.lat == query_call).all()
#     weather_data = []
#     for data in weather_container:
#         container = {
#             "city": data.city, 
#             "country": data.country, 
#             "region": data.region, 
#             "avgtemp": data.avgtemp, 
#             "date": data.date, 
#             "date_epoch": data.date_epoch, 
#             "maxtemp": data.maxtemp, 
#             "mintemp": data.mintemp, 
#             "sunhour": data.sunhour, 
#             "totalsnow": data.totalsnow, 
#             "uv_index": data.uv_index, 
#             "magnitude": data.magnitude, 
#             "place": data.place, 
#             "lat": data.lat, 
#             "long": data.long
#         }
#         weather_data.append(container)
#     return weather_data

In [ ]:
# latitude = '53.42'
# query_from_db_to_web = weatherTimeSeries(latitude)
# query_from_db_to_web

In [ ]:

dt3 = datetime.fromtimestamp(1597074974330 / 1000)
dt3.strftime("%B")

In [ ]:
dt3 = datetime.fromtimestamp(1597074974330 / 1000)
month_name = dt3.strftime("%B")
day = dt3.strftime("%d")
year = dt3.strftime("%Y")

month_day = month_name + " " + day
month_day_year = month_name + " " + day + ", " + year

date = {
    "month_day": month_day,
    "month_day_year": month_day_year,
}
date

In [ ]:
def spellDateUnix(datestring):
    dt3 = datetime.fromtimestamp(datestring / 1000)
    month_name = dt3.strftime("%B")
    day = dt3.strftime("%d")
    year = dt3.strftime("%Y")

    month_day = month_name + " " + day
    month_day_year = month_name + " " + day + ", " + year

    date = {
        "month_day": month_day,
        "month_day_year": month_day_year,
    }

    return date
spellDateUnix(1597074974330)

In [ ]:
def spellDate(datestring):
    date_time_obj = datetime.datetime.strptime(datestring, '%Y-%m-%d')
    month_name = date_time_obj.strftime("%B")
    day = date_time_obj.strftime("%d")
    year = date_time_obj.strftime("%Y")

    month_day = month_name + " " + day
    month_day_year = month_name + " " + day + ", " + year

    date = {
        "month_day": month_day,
        "month_day_year": month_day_year,
    }
    return date


def aboveSixQuake():
    Base = automap_base()
    Base.prepare(engine, reflect=True)
    # Check db table names
    # Base.classes.keys()
    weather_table = Base.classes.weatherSeries
    weather_container = session.query(weather_table).filter(weather_table.magnitude > 6).all()
    weather_facts = []
    magnitude_list = []
    count = 0
    for data in weather_container:
        count += 1
        
        # make a list of magnitudes recorded greater than 6 and get avg temp
        magnitude_list.append(data.avgtemp)
        magnitude = data.magnitude
        magnitude_keep = 6
        # Get highest recorded earthquake
        if data.magnitude > magnitude_keep:
            magnitude_keep = data.magnitude
            location = data.country
            city = data.city
            temp_low = data.mintemp
            temp_high = data.maxtemp
            avg_temp_at_time = data.avgtemp
            date = data.date
        else:
            continue
            
    # Get avgtemp from list        
    def Average(lst): 
        return sum(lst) / len(lst) 
    quake_avg = Average(magnitude_list)


    spell_dates = spellDate(date)
    
    container = {
        "count": count, 
        "avgtemp": quake_avg,
        "highest_magnitude": magnitude_keep, 
        "highest_city": city,
        "highest_location": location,
        "temp_low": temp_low,
        "temp_high": temp_high,
        "avg_temp_at_time": avg_temp_at_time,
        "date": spell_dates,
         
    }
    weather_facts.append(container)
    return weather_facts

In [ ]:
query_from_db_to_web = aboveSixQuake()
query_from_db_to_web

In [ ]:
## Convert Date to "August 10, 2020" and "August 10"

In [ ]:
import datetime

In [ ]:

import datetime

def spellDate(datestring):
    
    date_time_obj = datetime.datetime.strptime(datestring, '%Y-%m-%d')
    month_name = date_time_obj.strftime("%B")
    day = date_time_obj.strftime("%d")
    year = date_time_obj.strftime("%Y")

    month_day = month_name + " " + day
    month_day_year = month_name + " " + day + ", " + year
    
    date = {
        "month_day": month_day,
        "month_day_year": month_day_year,
    }
    return date


------

In [ ]:
# latlong_container = session.query(latlong_table).filter(latlong_table.lat == '').all()

In [ ]:
# for data in latlong_table:
#     print(f"Date: {data.lat}, Temp: {data.long}")

In [ ]:
# Join two tables in the db
# def join_example():
#     records = session.query(WeatherSeries).\
#         join(LatLong, LatLong.id == WeatherSeries.id).all()
# #     print(records)
#     for record in records:
#         recordObject = {
#             'city': record.city,
#             'country': record.country,
#             'date': record.date,
#             'maxtemp': record.maxtemp,
#             'latlong_rel': record.latlong_rel
#         }
#         print(recordObject)
# join_example()        

In [ ]:
# Create DataFrame from sql table Weather
# weather_data_df = pd.read_sql("SELECT * FROM LatLong", conn)
# weather_data_df

In [ ]:
### 
## KEEP !!
## DO NOT DELETE ALL THIS
###

# Create a Specific Instance of the "weather_data" and "earthquake_data" classes
# ----------------------------------
# Data needs to pass through this. A for loop while need to be created to consistantly send multiple files
# weather_data = WeatherSeries(
#     lat = '',
#     lon = '',
#     date = '',
#     city = '',
#     country = '',
#     region = '',
#     avgtemp = '',
#     date_epoch = '',
#     maxtemp = '',
#     mintemp = '',
#     sunhour = '',
#     totalsnow = '',
#     uv_index = '',
#     )

# earthquake_data = Earthquake(
#     lat = '',
#     lon = '',
#     date = '',
#     mag = '',
#     location = ''
#     )

-------------

-----------

In [ ]:
# def checkEqual3(lst):
#     return lst[1:] == lst[:-1]

# def checkRowValueDups(data):
#     dictOfKeys = data.keys()
#     for column in data:
#         column_values = data[column].to_list()
#         result = checkEqual3(column_values)
# #         print('passed.')
#     if result == True:
#         all_logs_df = all_logs.iloc[0:1, 0:]
#     else:
#         print("Couldnt complete cleaning. Different value found in column where only duplicates live. Check function checkRowValueDups() for information")
#     return all_logs_df

# all_logs_df = checkRowValueDups(all_logs)

In [ ]:
# all_logs_df['2020-08-03'][0]

In [ ]:
# stuff = (list(all_logs.itertuples(index=False, name=None)))